In [86]:
#Imports
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
import lasagne
from sklearn.cross_validation import train_test_split
from lasagne.updates import nesterov_momentum
from lasagne import layers
import theano.tensor as T
import theano

In [2]:
tempData = np.recfromcsv('letter.data', delimiter = "\t")


In [4]:
y = []
pos_arr = []
for i in range(0, len(tempData)):
    y.append(tempData[i][1])
    pos_arr.append(tempData[i][2])
len(y)

len(tempData)

52151

In [ ]:
# print tempData[110]
#tempArr = [[0 for i in xrange(7)] for j in xrange(15)]
batches = []
x = []
temp = []
for j in range(0, len(tempData) - 30001):
    for i in range(6,len(tempData[j])-1):
        temp += [tempData[j][i]]
    x.append(temp)
    temp = []
#reshaped = reshaped.astype('uint8')*255
len(x)


In [16]:
batches = []
x = []
temp = []
for j in range(len(x), 41245 ):
    for i in range(6,len(tempData[j])-1):
        temp += [tempData[j][i]]
    x.append(temp)
    temp = []
#reshaped = reshaped.astype('uint8')*255
x_batch2 = x
len(x_batch2)

41245

In [45]:
batch_1_start = 21245
x_batch1 = x[0:batch_1_start]
x_batch2 = x[batch_1_start:len(x)]
y_batch1 = y[0:batch_1_start]
y_batch2 = y[batch_1_start:len(x)]
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_batch1, y_batch1, test_size=0.33, random_state=42)
x_train2, x_test2, y_train2, y_test2 = train_test_split(x_batch2, y_batch2, test_size=0.33, random_state=42)

In [145]:
len(np.unique(y))

26

In [44]:
#Nearest Centroid
cent = NearestCentroid()
cent.fit(x_train1, y_train1)
cent.score(x_test1, y_test1)

0.61845671088289833

In [42]:
#SVM
clf2 = SVC(kernel='rbf') 
clf2.fit(x_train1, y_train1)
clf2.score(x_test1, y_test1)

0.80915703893881041

In [138]:
def build_mlp(input_var):
    l_in = lasagne.layers.InputLayer(shape=(None, 1, 16, 8),input_var=input_var)
    l_in_drop = lasagne.layers.DropoutLayer(l_in, p=0.2)
    l_hid1 = lasagne.layers.DenseLayer(l_in_drop, num_units=300,nonlinearity=lasagne.nonlinearities.rectify,
                                       W=lasagne.init.GlorotUniform())
    l_hid1_drop = lasagne.layers.DropoutLayer(l_hid1, p=0.5)
    l_hid2 = lasagne.layers.DenseLayer(l_hid1_drop, num_units=300,nonlinearity=lasagne.nonlinearities.rectify)
    l_hid2_drop = lasagne.layers.DropoutLayer(l_hid2, p=0.5)
    l_out = lasagne.layers.DenseLayer(l_hid2_drop, num_units=26,nonlinearity=lasagne.nonlinearities.softmax)
    return l_out


In [147]:
#MLP
input_var = T.tensor4('inputs')
target_var = T.lvector('targets')
network = build_mlp(input_var)
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.01, momentum=0.9)
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction, target_var)
test_loss = test_loss.mean()
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var), dtype=theano.config.floatX)
train_fn = theano.function([input_var, target_var], loss, updates=updates)
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])
train_error, train_acc, test_err, test_acc = 0, 0, 0, 0
y_reshaped, x_reshaped = [], []
for i in range(len(x_train1)):
    x_reshaped.append(np.asarray(x_train1[i]).reshape(-1, 1, 16, 8) / np.float32(256))
    y_reshaped.append(np.asarray([ord(y_train1[i])])/np.float32(256))
    train_fn(x_reshaped[i], y_reshaped[i])

TypeError: ('Bad input argument to theano function with name "<ipython-input-147-d18f94f57d7a>:14"  at index 1(0-based)', 'TensorType(int64, vector) cannot store a value of dtype float64 without risking loss of precision. If you do not mind this loss, you can: 1) explicitly cast your data to int64, or 2) set "allow_input_downcast=True" when calling "function".', array([ 0.43359375]))

In [111]:

np.asarray(x_train1[-1]).reshape(16,8).shape

(16, 8)

In [16]:
clf_poly = SVC(kernel='poly', degree=9)
clf_poly.fit(batch1, y_batch1)
clf_poly.fit(batch2, y_batch2)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=9, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [17]:
clf_poly.score(valid_set, y_cross_valid)

0.11720164609053497

In [26]:
neigh = KNeighborsClassifier(n_neighbors=20)
neigh.fit(batch2, y_batch2)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=20, p=2,
           weights='uniform')

In [25]:
neigh.score(batch1, y_batch1)

0.65859999999999996

ImportError: No module named blocks.bricks